In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import json
import requests
import io
import seaborn as sns
import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go


## dataset manipulate

In [57]:
raw_data = pd.read_csv("all_sports_data.csv")
raw_data.describe(),raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15560 entries, 0 to 15559
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       15560 non-null  object 
 1   overall    15560 non-null  int64  
 2   height_cm  15560 non-null  int64  
 3   weight_kg  15560 non-null  int64  
 4   Age        15560 non-null  int64  
 5   years_pro  2028 non-null   float64
 6   position   15560 non-null  object 
 7   APY        15560 non-null  float64
 8   sport      15560 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 1.1+ MB


(            overall     height_cm     weight_kg           Age    years_pro  \
 count  15560.000000  15560.000000  15560.000000  15560.000000  2028.000000   
 mean      67.139524    182.639203     79.429177     25.456298     3.856509   
 std        7.903444      7.823814     15.025329      4.650786     3.443049   
 min       26.000000    156.000000     49.000000     16.000000     0.000000   
 25%       62.000000    177.000000     71.000000     22.000000     1.000000   
 50%       67.000000    183.000000     76.000000     25.000000     3.000000   
 75%       72.000000    188.000000     83.000000     29.000000     6.000000   
 max       99.000000    226.000000    172.000000     43.000000    22.000000   
 
                 APY  
 count  1.556000e+04  
 mean   1.261962e+06  
 std    3.972747e+06  
 min    0.000000e+00  
 25%    5.200000e+04  
 50%    2.080000e+05  
 75%    9.360000e+05  
 max    6.000000e+07  ,
 None)

In [58]:
data_scaled = raw_data.copy()
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler()

for sport in ['NFL', 'NBA','FIFA']:
    sport_mask = data_scaled['sport'] == sport
    sport_indices = data_scaled[sport_mask].index
    sport_overall = data_scaled.loc[sport_mask, 'overall'].values.reshape(-1, 1)
    
    minmax = mm_scaler.fit_transform(sport_overall)
    data_scaled.loc[sport_indices, 'overall_minmax'] = minmax.flatten() 
    
    standard = ss_scaler.fit_transform(sport_overall)
    data_scaled.loc[sport_indices, 'overall_standard'] = standard.flatten()  


data_scaled

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,NFL,0.835616,1.582078
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,NFL,0.767123,1.082987
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,NFL,0.602740,-0.114831
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,NFL,0.945205,2.380623
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,NFL,0.780822,1.182805
...,...,...,...,...,...,...,...,...,...,...,...
15555,Umut Yaşar Keçeci,58,182,73,19,NaN,ST,104000.0,FIFA,0.250000,-1.150147
15556,Rakan Al Kaabi,54,174,71,20,NaN,"CDM, CM",104000.0,FIFA,0.159091,-1.702407
15557,Rodrigo Frutos,58,188,87,20,NaN,GK,26000.0,FIFA,0.250000,-1.150147
15558,Christian Bos,55,180,70,18,NaN,RB,26000.0,FIFA,0.181818,-1.564342


In [76]:
data_no_years = data_scaled.drop(labels="years_pro",  axis=1)
data_no_years 

,name,overall,height_cm,weight_kg,Age,position,APY,sport,overall_minmax,overall_standard
0,Dak Prescott,87,188,108,30,QB,60000000.0,NFL,0.835616,1.582078
1,Trevor Lawrence,82,198,100,23,QB,55000000.0,NFL,0.767123,1.082987
2,Jordan Love,70,193,99,24,QB,55000000.0,NFL,0.602740,-0.114831
3,Joe Burrow,95,193,98,26,QB,55000000.0,NFL,0.945205,2.380623
4,Tua Tagovailoa,83,185,98,25,QB,53100000.0,NFL,0.780822,1.182805
...,...,...,...,...,...,...,...,...,...,...
15555,Umut Yaşar Keçeci,58,182,73,19,ST,104000.0,FIFA,0.250000,-1.150147
15556,Rakan Al Kaabi,54,174,71,20,"CDM, CM",104000.0,FIFA,0.159091,-1.702407
15557,Rodrigo Frutos,58,188,87,20,GK,26000.0,FIFA,0.250000,-1.150147
15558,Christian Bos,55,180,70,18,RB,26000.0,FIFA,0.181818,-1.564342


In [59]:
data_pro_years = data_scaled[data_scaled["sport"].isin(["NFL", "NBA"])]
data_pro_years

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY,sport,overall_minmax,overall_standard
0,Dak Prescott,87,188,108,30,7.0,QB,60000000.0,NFL,0.835616,1.582078
1,Trevor Lawrence,82,198,100,23,2.0,QB,55000000.0,NFL,0.767123,1.082987
2,Jordan Love,70,193,99,24,3.0,QB,55000000.0,NFL,0.602740,-0.114831
3,Joe Burrow,95,193,98,26,3.0,QB,55000000.0,NFL,0.945205,2.380623
4,Tua Tagovailoa,83,185,98,25,3.0,QB,53100000.0,NFL,0.780822,1.182805
...,...,...,...,...,...,...,...,...,...,...,...
2023,Zach Edey,79,226,138,22,1.0,C,5756880.0,NBA,0.366667,0.131788
2024,Zach LaVine,85,196,91,29,11.0,SG,43031940.0,NBA,0.566667,1.123926
2025,Zeke Nnaji,72,206,109,23,5.0,C,8888889.0,NBA,0.133333,-1.025706
2026,Ziaire Williams,76,206,84,22,4.0,SF,6133005.0,NBA,0.266667,-0.364281


In [77]:
nba_year = data_pro_years[ data_pro_years["sport"] == "NBA"] 
nfl_year = data_pro_years[ data_pro_years["sport"] == "NFL"]
nba = data_no_years [ data_no_years ["sport"] == "NBA"] 
nfl =data_no_years[ data_no_years["sport"] == "NFL"]
fifa = data_no_years [ data_no_years ["sport"] == "FIFA"] 

In [130]:
dataset = [
    (nba_year, "nba_year"),
    (nfl_year, "nfl_year"),
    (nba, "nba"),
    (nfl, "nfl"),
    (fifa, "fifa")
]

## model

In [144]:
def APY_linear_regression_model(dataset, overall_type="overall", years=None):
    
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[[overall_type, "height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[[overall_type, "height_cm", "weight_kg", "Age"]] 
        
    y = dataset["APY"]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    lin_model = LinearRegression()
    lin_model.fit(x_train, y_train)
    
    y_pred = lin_model.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results(with overall):")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    

In [176]:
def APY_without_overall_linear_regression_model(dataset, years=None):
    
    dataset_name = dataset.__name__ if hasattr(dataset, "__name__") else str(dataset)
    has_years_pro = "years_pro" in dataset.columns
    
    if has_years_pro:
        x = dataset[[ "height_cm", "weight_kg", "Age", "years_pro"]] 
    else:
        x = dataset[[ "height_cm", "weight_kg", "Age"]] 
        
    y = dataset["APY"]
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2025, test_size=0.2)
    
    lin_model = LinearRegression()
    lin_model.fit(x_train, y_train)
    
    y_pred = lin_model.predict(x_test) 
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Results(without overall):")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    

In [147]:
model = [APY_with_overall_linear_regression_model, APY_without_overall_linear_regression_model]

## run model

In [177]:
def run_model(datasets_with_names, models):
    results = []

    for dataset, dataset_name in datasets_with_names:

        for model_func in models:
            print(f"Running {model_func.__name__} on {dataset_name}")
            try:
                model_result = model_func(dataset)
                results.append({
                    'dataset': dataset_name,
                    'model': model_func.__name__,
                    'results': model_result
                })
                print("-" * 50)
            except Exception as e:
                print(f"Error running {model_func.__name__} on {dataset_name}: {e}")

In [178]:
run_model(dataset, model)

Running APY_with_overall_linear_regression_model on nba_year
Model Results(with overall):
MSE: 48698969825719.6562
RMSE: 6978464.7184
R²: 0.5981
--------------------------------------------------
Running APY_without_overall_linear_regression_model on nba_year
Model Results(without overall):
MSE: 101907847455120.7656
RMSE: 10094941.6767
R²: 0.1590
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl_year
Model Results(with overall):
MSE: 18883384903311.2344
RMSE: 4345501.6860
R²: 0.3735
--------------------------------------------------
Running APY_without_overall_linear_regression_model on nfl_year
Model Results(without overall):
MSE: 24962081256429.3203
RMSE: 4996206.6867
R²: 0.1719
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nba
Model Results(with overall):
MSE: 48241489167503.4609
RMSE: 6945609.3446
R²: 0.6019
--------------------------------------------------
Running APY

In [179]:
def run_model_check(datasets_with_names, model_func, **model_kwargs):
    results = []
    for dataset, dataset_name in datasets_with_names:
        print(f"Running {model_func.__name__} on {dataset_name}")

        try:
            model_result = model_func(dataset, **model_kwargs)
            results.append({
                'dataset': dataset_name,
                'model': model_func.__name__,
                'results': model_result
            })
            
            print("-" * 50)
        except Exception as e:
            print(f"Error running {model_func.__name__} on {dataset_name}: {e}")
    

In [180]:
run_model_check(datasets, APY_with_overall_linear_regression_model, overall_type="overall_standard")

Running APY_with_overall_linear_regression_model on nba_year
Model Results(with overall):
MSE: 48698969825719.6875
RMSE: 6978464.7184
R²: 0.5981
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl_year
Model Results(with overall):
MSE: 18883384903311.2539
RMSE: 4345501.6860
R²: 0.3735
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nba
Model Results(with overall):
MSE: 48241489167503.4844
RMSE: 6945609.3446
R²: 0.6019
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl
Model Results(with overall):
MSE: 19441712628964.9336
RMSE: 4409275.7488
R²: 0.3550
--------------------------------------------------
Running APY_with_overall_linear_regression_model on fifa
Model Results(with overall):
MSE: 694354464833.3361
RMSE: 833279.3438
R²: 0.3889
--------------------------------------------------


In [181]:
run_model_check(datasets, APY_with_overall_linear_regression_model, overall_type="overall_minmax")

Running APY_with_overall_linear_regression_model on nba_year
Model Results(with overall):
MSE: 48698969825719.6719
RMSE: 6978464.7184
R²: 0.5981
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl_year
Model Results(with overall):
MSE: 18883384903311.2305
RMSE: 4345501.6860
R²: 0.3735
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nba
Model Results(with overall):
MSE: 48241489167503.4531
RMSE: 6945609.3446
R²: 0.6019
--------------------------------------------------
Running APY_with_overall_linear_regression_model on nfl
Model Results(with overall):
MSE: 19441712628964.9648
RMSE: 4409275.7488
R²: 0.3550
--------------------------------------------------
Running APY_with_overall_linear_regression_model on fifa
Model Results(with overall):
MSE: 694354464833.3359
RMSE: 833279.3438
R²: 0.3889
--------------------------------------------------
